<a href="https://colab.research.google.com/github/PetraB42/DSpracticum2020/blob/main/Tweet_advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

used surces: tensorflow.org/recommenders/examples/quickstart

### Import TFRS

First, install and import TFRS:

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import pandas as pd

import tensorflow_recommenders as tfrs

In [3]:
url = 'https://raw.githubusercontent.com/simecek/dspracticum2020/5990195ac6f7c78c1a4d944e70b28a42b96b3ec3/lecture_09/assignment/fake_v1_100x252.csv'
a = pd.read_csv(url)


### Read the data

data preprocessing

In [4]:
#reshape data into usable format
a.rename(columns={ a.columns[0]: "user_ID" }, inplace = True)

# prepare rating per use data
ratings = pd.melt(a, id_vars='user_ID')
ratings.rename(columns={ratings.columns[1]: "tweet" }, inplace = True)
ratings.rename(columns={ratings.columns[2]: "rating" }, inplace = True)
ratings = ratings[ratings['rating'].notna()] # clear rating data so that they contain only rated tweets per user
ratings = ratings.reset_index(drop=True)

#prepare average rating per tweet
tweets = a.drop(columns=["user_ID"])
tweets = pd.DataFrame(tweets.mean(axis=0))
tweets.reset_index(level=0, inplace=True)
tweets.rename(columns={tweets.columns[0]: "tweet" }, inplace = True)
tweets.rename(columns={tweets.columns[1]: "avr_rating" }, inplace = True)
tweets = tweets.sort_values(by = "avr_rating", ascending = False)
tweets = tweets.reset_index(drop=True)


#check numbers of user and indexses

num_tweets_rat = ratings.tweet.nunique()
num_users = ratings.user_ID.nunique()
num_tweets = tweets.tweet.nunique()


print("USERS: {} TOTAL TWEETS: {} RATED TWEETS: {}".format(num_users, num_tweets, num_tweets_rat))

USERS: 100 TOTAL TWEETS: 252 RATED TWEETS: 252


In [5]:
import tempfile
# turn dataframe into tensorflow object
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))
tweets = tf.data.Dataset.from_tensor_slices(dict(tweets))

# Select the basic features.
ratings = ratings.map(lambda x: {
    "tweet": x["tweet"],
    "user_id": x["user_ID"],
    "rating": x["rating"]
  })

tweets = tweets.map(lambda x: x["tweet"])


In [6]:
#Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [7]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

tweets_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
tweets_vocabulary.adapt(tweets)

## Define a model


We can define a TFRS model by inheriting from tfrs.Model and implementing the compute_loss method:

In [8]:
class TweetLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      tweet_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and tweet representations.
    self.user_model = user_model
    self.tweet_model = tweet_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    tweet_embeddings = self.tweet_model(features["tweet"])

    return self.task(user_embeddings, tweet_embeddings, compute_metrics=not training)

Define the two models and the retrieval task.

In [9]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 32)
])
tweet_model = tf.keras.Sequential([
    tweets_vocabulary,
    tf.keras.layers.Embedding(tweets_vocabulary.vocab_size(), 32)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    candidates = tweets.batch(128).cache().map(tweet_model)
  )
)


### Fit and evaluate it.

Create the model, train it, and generate predictions:



In [11]:
# Create a retrieval model.
model = TweetLensModel(user_model, tweet_model,task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(3000).cache()

# Train for 3 epochs.
model.fit(cached_train, epochs=8)

Epoch 1/8
2/2 [==============================] - 0s 7ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 18143.4049 - regularization_loss: 0.0000e+00 - total_loss: 18143.4049
Epoch 2/8
2/2 [==============================] - 0s 5ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 18091.0840 - regularization_loss: 0.0000e+00 - total_loss: 18091.0840
Epoch 3/8
2/2 [==============================] - 0s 5ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_to

In [12]:
#cached_test = test.shuffle(100_000).cache()

#model.evaluate(test.batch(4096), return_dict=True)

In [13]:
model.evaluate(ratings.batch(5000), return_dict=True)

1/1 [==============================] - 0s 2ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0138 - factorized_top_k/top_5_categorical_accuracy: 0.0944 - factorized_top_k/top_10_categorical_accuracy: 0.1906 - factorized_top_k/top_50_categorical_accuracy: 0.7610 - factorized_top_k/top_100_categorical_accuracy: 0.9496 - loss: 38468.3672 - regularization_loss: 0.0000e+00 - total_loss: 38468.3672


{'factorized_top_k/top_100_categorical_accuracy': 0.9495999813079834,
 'factorized_top_k/top_10_categorical_accuracy': 0.19059999287128448,
 'factorized_top_k/top_1_categorical_accuracy': 0.013799999840557575,
 'factorized_top_k/top_50_categorical_accuracy': 0.7609999775886536,
 'factorized_top_k/top_5_categorical_accuracy': 0.09440000355243683,
 'loss': 38468.3671875,
 'regularization_loss': 0,
 'total_loss': 38468.3671875}

In [14]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(tweets.batch(100).map(model.tweet_model), tweets)

# Get some recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Top 3 recommendations for user 42: [b'tweet157' b'tweet68' b'tweet110']
